In [6]:
import numpy as np
import cv2
import PIL.Image as Image
import pathlib, os, random, math
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import pathlib
os.environ['DIR'] = "C:\\Users\\kunta\\Dropbox\\abc\\Automated-Shopping-Cart\\Code So Far"


In [7]:

IMAGE_SHAPE = (224, 224)
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [8]:
train_dir="..\\dataset\\train"
train_dir = pathlib.Path(train_dir)
train_dir

WindowsPath('../dataset/train')

In [21]:
val_dir="..\\dataset\\val"
val_dir = pathlib.Path(val_dir)
val_dir

WindowsPath('../dataset/val')

In [20]:
test_dir="..\\dataset\\test"
test_dir = pathlib.Path(test_dir)
test_dir

WindowsPath('../dataset/test')

In [22]:
epochs = 30
batch_size = 32
img_height, img_width = 224,224
input_shape = (img_height, img_width, 3)
data_gen = ImageDataGenerator(rescale=1.0 / 255)

train_ds = data_gen.flow_from_directory(str(train_dir)+"\\fruits", target_size=(img_height, img_width),class_mode='sparse', batch_size=batch_size, subset='training')
val_ds = data_gen.flow_from_directory(str(val_dir)+"\\fruits", target_size=(img_height, img_width),class_mode='sparse', batch_size=batch_size, shuffle=False)
test_ds = data_gen.flow_from_directory(str(test_dir)+"\\fruits", target_size=(img_height, img_width),class_mode='sparse', batch_size=batch_size, shuffle=False)

Found 9370 images belonging to 13 classes.
Found 1649 images belonging to 13 classes.
Found 2990 images belonging to 13 classes.


In [19]:
class BaseModel(tf.keras.Model):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.C1 = Conv2D(32, (3 * 3), padding='same', input_shape=input_shape)
        self.B1 = BatchNormalization()
        self.A1 = Activation('relu')
        self.P1 = MaxPooling2D(2, padding='same')
        
        self.C2 = Conv2D(32, (3 * 3), padding='same')
        self.B2 = BatchNormalization()
        self.A2 = Activation('relu')
        self.P2 = MaxPooling2D(2, padding='same')
        self.Dr1 = Dropout(0.2)

        self.C3 = Conv2D(64, (3 * 3), padding='same')
        self.B3 = BatchNormalization()
        self.A3 = Activation('relu')
        self.P3 = MaxPooling2D(2, padding='same')
        self.Dr2 = Dropout(0.3)
        
        self.F1 = Flatten()
        self.D1 = Dense(256, activation='relu')
        self.B4 = BatchNormalization()
        self.D2 = Dense(256, activation='relu')
        self.D3 = Dense(256, activation='relu')
        self.D4 = Dense(36, activation='softmax')

    def call(self, x):
        x = self.C1(x)
        x = self.B1(x)
        x = self.A1(x)
        x = self.P1(x)
        
        x = self.C2(x)
        x = self.B2(x)
        x = self.A2(x)
        x = self.P2(x)
        x = self.Dr1(x)
        
        x = self.C3(x)
        x = self.B3(x)
        x = self.A3(x)
        x = self.P3(x)
        x = self.Dr2(x)
        
        x = self.F1(x)
        x = self.D1(x)
        x = self.B4(x)
        x = self.D2(x)
        x = self.D3(x)
        y = self.D4(x)
        return y
    def __repr__(self):
        return 'Huang_Model'

net = BaseModel()

net.compile(optimizer='adam',
           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
           metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = './Model.ckpt'
if os.path.exists(checkpoint_save_path + '.idnex'):
    net.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path, save_best_only=True, save_weights_only=True)    

history = net.fit(train_ds, epochs=2, batch_size=batch_size, validation_data=val_ds, callbacks=[cp_callback])

net.summary()

file = open('./Weights.txt', 'w')
for v in net.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

Epoch 1/2
293/293 [==============================] - 2106s 7s/step - loss: 1.8259 - sparse_categorical_accuracy: 0.3954 - val_loss: 2.0473 - val_sparse_categorical_accuracy: 0.3367
Epoch 2/2
293/293 [==============================] - 1843s 6s/step - loss: 1.5470 - sparse_categorical_accuracy: 0.4832 - val_loss: 2.5675 - val_sparse_categorical_accuracy: 0.2527
Model: "base_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           multiple                  7808      
                                                                 
 batch_normalization_2 (Batc  multiple                 128       
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   multiple                  0         
                                                                 
 max_pooling2d_2 (MaxP

In [23]:
net.evaluate(test_ds)

94/94 [==============================] - 72s 771ms/step - loss: 2.9784 - sparse_categorical_accuracy: 0.1629


[2.978409767150879, 0.16287624835968018]